<a href="https://colab.research.google.com/github/institutohumai/cursos-python/blob/master/PracticasDeDesarrollo/3_Desarrollo_III/1_Setuptools/clase_03_setuptools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab" data-canonical-src="https://colab.research.google.com/assets/colab-badge.svg"></a>

# 1. Setuptools: Como crear y distribuir tus propios paquetes
## 1.1. Introducción
¿Cómo se crea un paquete personalizado en Python?  
¿Cómo podemos compartirlo junto con sus dependencias?  
¿Se puede instalar un paquete con pip desde un repositorio de Git?  

En esta sección vamos a aprender a:
- Entender los requerimientos de un paquete de Python.
- Construir un paquete desde cero o transformar un proyecto existente en un paquete.
- Hacer el paquete instalable con pip desde un repositorio.
- Actualizar un paquete.

## 1.2. Empaquetando el código
![image](./images/empaquetando.jpg)  
Supongamos que tenemos una serie de funciones utilitarias que queremos compartir con nuestro equipo o con la comunidad. Lo primero que vamos a hacer es crear un paquete con ellas. Éste paso es necesario si queremos usar `pip` para la instalación.
Vamos a tomar como ejemplo una serie de funciones genéricas que se encuentran alojadas en el archivo `functions.py`

In [ ]:
%%writefile functions.py
# No usamos este paquete pero igual lo importamos para generar una dependencia.
import requests

def saludar_comunidad():
	print('Hola comunidad de Humai')

def informar_clase():
	print('La siguiente clase es el lunes')

In [11]:
!cat functions.py

# No usamos este paquete pero igual lo importamos para generar una dependencia.
import requests

def saludar_comunidad():
	print('Hola comunidad de Humai')

def informar_clase():
	print('La siguiente clase es el lunes')

1- Creamos una carpeta con el nombre que vamos a ponerle a nuestro paquete, en este caso se va a llamar `humai_utils` Le agregamos una serie de archivos al mismo:
- `humai_utils/functions.py` En este archivo se van a guardar las funciones que queremos compartir, en este caso contiene dos: `saludar_comunidad()` e `informar_clase()`
- `humai_utils/__init__.py` Esto le indica a Python que la carpeta `humai_utils` es un paquete, este archivo también nos habilita a importar funciones individualmente de la forma `import saludar_comunidad from humai_utils` en lugar de `from humai_utils.functions import saludar_comunidad`. Para ello tenemos que colocar en la primera linea:  
`from .archivo-con-las-funciones import funcion1, funcion2, etc.`

In [12]:
!cat humai_utils/__init__.py

from .functions import informar_clase, saludar_comunidad

El archivo `__init__.py` es requerido pero el contenido es opcional, puede estar vacío.

2- En la raíz del proyecto creamos un archivo con el nombre `setup.py` con el siguiente contenido:

In [ ]:
%%writefile setup.py
import setuptools

with open("DESCRIPTION.md", "r", encoding="utf-8") as fh:
    long_description = fh.read()

setuptools.setup(
    name='humai_utils',
    version='0.1.0',
    author='Ramiro Savoie',
    author_email='ramiro@deployr.ai',
    description='A compilation of Humai utility functions',
    long_description=long_description,
    long_description_content_type="text/markdown",
    url='https://github.com/institutohumai/humai_utils',
    project_urls = {
        "Bug Tracker": "https://github.com/institutohumai/humai_utils/issues"
    },
    license='MIT',
    packages=['humai_utils'],
    install_requires=['requests'],
)

In [13]:
!cat setup.py

import setuptools

with open("DESCRIPTION.md", "r", encoding="utf-8") as fh:
    long_description = fh.read()

setuptools.setup(
    name='humai_utils',
    version='0.1.0',
    author='Ramiro Savoie',
    author_email='ramiro@deployr.ai',
    description='A compilation of Humai utility functions',
    long_description=long_description,
    long_description_content_type="text/markdown",
    url='https://github.com/institutohumai/humai_utils',
    project_urls = {
        "Bug Tracker": "https://github.com/institutohumai/humai_utils/issues"
    },
    license='MIT',
    packages=['humai_utils'],
    install_requires=['requests'],
)

Este archivo le indica a `pip` que necesita nuestro paquete para poder ser instalado. Analicémoslo línea por línea:  
- En la primera linea importamos `setuptools`, que es el paquete que permite generar otros paquetes.
- A continuación con `with` se abre un archivo con una descripción detallada de lo que hace nuestro paquete y la carga en una variable, la cual se usa más abajo.
- La función `.setup()` del paquete `setuptools` recibe varios parametros que describen nuestro paquete:
    - Parámetro `name`: Le da nombre al paquete y debe coincidir con el nombre de la carpeta.
    - Parámetro `version`: Número de versión del paquete, `pip` lo usa para saber si el paquete instalado debe ser actualizado. Hay que incrementarlo si queremos que los usuarios actualicen.
    - Parámetro `long_description`: aquí utilizamos el contenido del archivo abierto previamente.
    - Parámetro `long_description_content_type`: Indica el formato de la descripción detallada, en este caso Markdown.
    - Parámetro `url`: La URL del repositorio de código del paquete.
    - Parámetro `project_urls`: Otros links de referencia del proyecto, como por ejemplo, en donde reportar bugs.
    - Parámetro `license`: Licencia con la cual se puede usar y distribuir el paquete, en este caso, MIT. Algunos otros ejemplos pueden encontrarse en [choosealicense.com](choosealicense.com)
    - Parámetro `packages`: Lista de todos los paquetes a construir, debe coincidir con el nombre de carpeta del paquete.
    - Parámetro `install_requires`: Listado de dependencias sobre las que construimos nuestro paquete. `pip` las va a instalar antes para que nuestro paquete pueda usarlas.

3- Agregamos el archivo `DESCRIPTION.md` con la descripcion completa del paquete y podría agregarse un archivo LICENSE con la licencia elegida. Son simplemente archivos de texto que no son siempre requeridos. 

In [ ]:
%%writefile DESCRIPTION.md
# Humai Utils

Este es un paquete con muchas funciones comunes que encontramos en nuestros proyectos y que queremos compartir con la comunidad.

In [14]:
!cat DESCRIPTION.md

# Humai Utils

Este es un paquete con muchas funciones comunes que encontramos en nuestros proyectos y que queremos compartir con la comunidad.

4- Con nuestro flamante paquete ya descripto por el `setup.py`, y en caso de que no lo hubieramos hecho, podemos iniciar un repositorio local con el comando `git init`.  
Añadimos un archivo `.gitignore` para no versionar en el repositorio archivos innecesarios que hagan crecer su tamaño.  
Si se trabajó dentro de un virtual environment también hay que ignorar su carpeta.

![image](./images/git-init.png)

Generalmente los paquetes de Python están compuestos de módulos y funciones, no suele haber notebooks de Jupyter. En este caso la incluimos solo para fines didácticos. Una vez que commiteamos la primera versión de nuestro paquete, pasemos a ver a como distribuirlo.  
![image](./images/primer-paquete.png)  
## 1.3. Distribuyendo el código vía GitHub
![image](./images/delivery.jpg)  
Lo que tenemos que hacer ahora es crear el repositorio en cualquier plataforma que soporte Git como GitHub o BitBucket.
![image](./images/primer-repo.png)  
Generalmente se le coloca al repositorio el mismo nombre que el paquete.  
A continuación debemos indicar la visibilidad del paquete, si va a permanecer privado en nuestra cuenta o nuestra organizacion o si va a poder ser instalado por cualquier usuario. Una breve descripción tambien ayuda a que cualquier persona que se encuentre con el paquete pueda tener una idea de para que sirve.  
![image](./images/descripcion.png)  
De las opciones a continuación no necesitamos ninguna porque ya nos encargamos previamente: ya agregamos un `.gitignore`, ya seleccionamos una licencia y ya escribimos un `README.md`. Le damos click al botón "Create Repository"  
![image](./images/crear-repo.png)  
Ya tenemos nuestro repositorio creado! Ahora entre las tres opciones que nos da GitHub, tenemos que usar la segunda, dado que nosotros ya tenemos un repositorio existente, en nuestra máquina local. Lo que tenemos que hacer ahora es "enlazarlos".  
En la terminal en la que corrimos el `git commit` corremos el comando que nos indica (seguramente los nombres van a ser distintos):  
![image](./images/repositorio-existente.png)  
Este comando no nos da ninguna respuesta cuando lo ejecutamos (¿podria no?), pero funcionó.  
![image](./images/remote-origin.png)  
Lo que hace a continuación es generar un nuevo branch llamado `main`, puede que ya estemos trabajando en el mismo, en ese caso no va a hacer nada:  
![image](./images/branch-main.png)  
Y finalmente hacemos un `git push` desde nuestro repositorio hacia el de GitHub:  
![image](./images/primer-push.png)  
Felicitaciones! Publicaste tu primer paquete de Python para que el mundo pueda maravillarse con tu código.  
Si volvemos a entrar al repositorio de Github vamos a ver el código de nuestro paquete.  
![image](./images/primer-paquete-publicado.png)  
Y ahora, como hacemos para instalarlo y usarlo en otro proyecto?  
## 1.3 Utilizando nuestro paquete en otro proyecto
Vamos a necesitar la URL de clonado de nuestro repo, no la que sale en el navegador, ésta:  
![image](./images/url-repo-https.png)  
Para mantenerlo simple vamos a usar la opción con https. Copiamos la URL del repositorio y nos dirigimos hacia el proyecto que quiere utilizar nuestro paquete, por supuesto abrimos una nueva terminal ahí.  
![image](./images/usando-nuestro-paquete.png)  
En nuestro proyecto, llamado `usando-nuestro-paquete` para que no queden dudas, vamos a instalar y usar nuestro nuevo paquete. Tenemos dos formas de hacerlo:
### 1.3.1 Instalando con pip en la terminal
La primera es instalarlo con `pip`, en este caso el nombre de nuestro paquete va a ser la URL de clonado de nuestro repositorio, pero agregando el prefijo `git+` a la URL de esta manera **git+**https://github.com/deployr-ai/humai_utils.git  
Quedando entonces así:  
`pip install git+https://github.com/deployr-ai/humai_utils.git`  
Cuando lo corramos `pip` va a clonar el repositorio por nosotros, va a leer el archivo `setup.py` y va a instalar las dependencias que le indicamos:  
![image](./images/build-local.png)  
En la última línea nos confirma que nuestro paquete fue instalado, con versión y todo!  
### 1.3.2 Agregándolo como dependencia
La segunda forma de usarlo es dentro del archivo `requirements.txt` junto con las otras dependencias. En este caso la URL que tenemos que agregar es la misma que en el caso anterior.  
Aqui podemos ver un ejemplo en conjunto con otra dependencia `boto3`

In [5]:
!cat usando-nuestro-paquete/requirements.txt

pandas
git+https://github.com/deployr-ai/humai_utils.git

En este caso para instalar usaríamos `pip install -r requirements.txt` y el resultado sería el mismo:
![image](./images/con-pandas.png)
Pequeño disclaimer: En ambos casos estamos instalando nuestro paquete en el **ambiente global**, lo cual no es una buena práctica, lo correcto es trabajarlo, como vimos en la clase anterior, con virtual environment, con conda o con Poetry. Gracias por su atención.

## 1.4 Uso
Para poder utilizar nuestro nuevo paquete instalado, se importa como cualquier otro paquete del sistema con `import humai_utils` o `from humai_utils import saludar_comunidad`

In [6]:
!cat usando-nuestro-paquete/main.py

# Importando nuestro propio paquete
from humai_utils import saludar_comunidad

# Llamando a una de sus funciones
saludar_comunidad()

## 1.5 Desinstalación
Y para desinstalarlo? En ese caso usamos `pip uninstall humai_utils`, sin toda la URL larga:  
![image](./images/desinstalacion.png)  
Nos pide confirmación por supuesto.

## 1.6 Actualizando el paquete
Si uno de nuestros compañeros agrega una nueva función y decide commitearla a nuestro repositorio, podemos usar `pip` para actualizar el paquete. Cada vez que se haga la llamada:
`pip install git+**https://github.com/deployr-ai/humai_utils.git`
`pip` va a chequear la versión en el archivo `setup.py` local contra la del repositorio, si hay un incremento de la misma, se va a actualizar la versión local.

## 1.7 Conclusiones
Como hemos podido ver, combinando el poder de empaquetado de Python junto con Git tenemos las siguientes ventajas:
- Fácil distribución, instalación y actualización desde una sola fuente central (una fuente de verdad)
- Control de versiones de nuestro paquete y poder colaborar con otros desarrolladores.
- Actualizar el paquete cuando se modifica y que nuestros usuarios puedan ser notificados del cambio.
- Poder instalar con pip y actualizar paquetes desde un repositorio privado.